In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from PIL import ImageSequence
import os

In [2]:
our_frames_no_grid = '../../output/tests/gif_ablation_study/our_frames_no_grid/SD/gif_stats.csv'
chatgpt_frames_no_grid = '../../output/tests/gif_ablation_study/chatgpt_frames_no_grid/SD/gif_stats.csv'
chatgpt_frames_grid = '../../output/tests/gif_ablation_study/chatgpt_frames_grid/SD/gif_stats.csv'

data_our_frames_no_grid = pd.read_csv(our_frames_no_grid)
data_chatgpt_frames_no_grid = pd.read_csv(chatgpt_frames_no_grid)
data_chatgpt_frames_grid = pd.read_csv(chatgpt_frames_grid)

# Add model names to each dataset
data_our_frames_no_grid['model_name'] = 'our_frames_no_grid'
data_chatgpt_frames_no_grid['model_name'] = 'chatgpt_frames_no_grid'
data_chatgpt_frames_grid['model_name'] = 'chatgpt_frames_grid'

# Select relevant columns
columns = ['model_name', 'prompt', 'clip', 'ms_ssim', 'lpips', 'temporal_consistency_loss']
model_data_our_frames_no_grid = data_our_frames_no_grid[columns]
model_data_chatgpt_frames_no_grid = data_chatgpt_frames_no_grid[columns]
model_data_chatgpt_frames_grid = data_chatgpt_frames_grid[columns]

# Concatenate all model data
all_model_data = pd.concat([ model_data_our_frames_no_grid, model_data_chatgpt_frames_no_grid, model_data_chatgpt_frames_grid], ignore_index=True)

# Display the combined DataFrame
print(all_model_data)


              model_name                                             prompt  \
0     our_frames_no_grid  A flower blooming from a bud to full bloom ove...   
1     our_frames_no_grid  A cat chasing a laser pointer dot across the r...   
2     our_frames_no_grid                A rotating 3D cube changing colors.   
3     our_frames_no_grid  A sunrise over the mountains turning into dayt...   
4     our_frames_no_grid   A person morphing into a wolf under a full moon.   
..                   ...                                                ...   
195  chatgpt_frames_grid              A snake slithering through the grass.   
196  chatgpt_frames_grid          A paintbrush changing colors as it moves.   
197  chatgpt_frames_grid              A volcano erupting with lava flowing.   
198  chatgpt_frames_grid                            An eye blinking slowly.   
199  chatgpt_frames_grid                     A paper crumpling into a ball.   

         clip   ms_ssim     lpips  temporal_consist

In [3]:
# Calculate mean values for each metric grouped by model_name
metric_columns = ['clip', 'ms_ssim', 'lpips', 'temporal_consistency_loss']
avg_metrics = all_model_data.groupby('model_name')[metric_columns].mean()

# Display the average metrics
print("\nAverage metrics by model:")
print(avg_metrics)


Average metrics by model:
                            clip   ms_ssim     lpips  \
model_name                                             
chatgpt_frames_grid     0.275563  0.588118  0.384020   
chatgpt_frames_no_grid  0.259140  0.131995  0.723372   
our_frames_no_grid      0.268245  0.123528  0.685848   

                        temporal_consistency_loss  
model_name                                         
chatgpt_frames_grid                      0.157294  
chatgpt_frames_no_grid                   0.389152  
our_frames_no_grid                       0.403228  


In [4]:
# Step 1: Normalize each metric (0-1 scaling)
all_model_data['ms_ssim_norm'] = 1 - ((all_model_data['ms_ssim'] - all_model_data['ms_ssim'].min()) / (all_model_data['ms_ssim'].max() - all_model_data['ms_ssim'].min()))
all_model_data['lpips_norm'] = (all_model_data['lpips'] - all_model_data['lpips'].min()) / (all_model_data['lpips'].max() - all_model_data['lpips'].min())
all_model_data['temporal_consistency_loss_norm'] = (all_model_data['temporal_consistency_loss'] - all_model_data['temporal_consistency_loss'].min()) / (all_model_data['temporal_consistency_loss'].max() - all_model_data['temporal_consistency_loss'].min())

# Step 2: Compute combined normalized loss
all_model_data['combined_loss'] = all_model_data['ms_ssim_norm'] + all_model_data['lpips_norm'] + all_model_data['temporal_consistency_loss_norm']

# Calculate mean values and standard deviations for each metric grouped by model_name
metric_columns = ['clip', 'ms_ssim', 'lpips', 'temporal_consistency_loss', 'combined_loss']
avg_metrics = all_model_data.groupby('model_name')[metric_columns].agg(['mean', 'std'])

# Format the output with ± and 3 decimal places
formatted_metrics = pd.DataFrame(index=avg_metrics.index)
for metric in metric_columns:
    formatted_metrics[metric] = avg_metrics[metric]['mean'].round(3).astype(str) + ' ± ' + avg_metrics[metric]['std'].round(3).astype(str)

# Display the average metrics
print("\nAverage metrics by model:")
print(formatted_metrics)


Average metrics by model:
                                 clip        ms_ssim          lpips  \
model_name                                                            
chatgpt_frames_grid     0.276 ± 0.022  0.588 ± 0.129  0.384 ± 0.094   
chatgpt_frames_no_grid  0.259 ± 0.021  0.132 ± 0.052  0.723 ± 0.043   
our_frames_no_grid      0.268 ± 0.026  0.124 ± 0.082  0.686 ± 0.062   

                       temporal_consistency_loss  combined_loss  
model_name                                                       
chatgpt_frames_grid                0.157 ± 0.062  0.819 ± 0.391  
chatgpt_frames_no_grid             0.389 ± 0.046  2.314 ± 0.156  
our_frames_no_grid                 0.403 ± 0.078  2.294 ± 0.252  


In [5]:
 # Get our frames with grid from main results

In [6]:
def generate_gif_figure(gif_path, n=4, save_name=None, spacing=True):
    gif = Image.open(gif_path)
    frames = [frame.copy() for frame in ImageSequence.Iterator(gif)]

    total_frames = len(frames)
    indices = np.linspace(0, total_frames - 1, n, dtype=int)

    # Create a figure with n equally spaced frames
    fig, axes = plt.subplots(1, n, figsize=(3 * n, 5))
    for ax, idx in zip(axes, indices):
        ax.imshow(frames[idx])
        ax.axis('off')

    if not spacing:
        plt.subplots_adjust(wspace=0, hspace=0)

    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

    if save_name is not None:
        fig.savefig(save_name, bbox_inches='tight', pad_inches=0)
    else:
        plt.show()

    plt.close(fig)

In [7]:
our_frames_grid = '../../output/tests/gif_prev_work_comparison/SD/a_cup_of_coffee_being_poured_w_f24_b3_g11.0_ip0.0_VideoText_previous.gif'
chatgpt_frames_grid = '../../output/tests/gif_ablation_study/chatgpt_frames_grid/SD/a_cup_of_coffee_being_poured_w_f24_b3_g11.0_ip0.0_VideoText_previous.gif'
chatgpt_frames_no_grid = '../../output/tests/gif_ablation_study/chatgpt_frames_no_grid/SD/a_cup_of_coffee_being_poured_w_f24_b3_g11.0_ip0.0_VideoText_previous.gif'
our_frames_no_grid = '../../output/tests/gif_ablation_study/our_frames_no_grid/SD/a_cup_of_coffee_being_poured_w_f24_b3_g11.0_ip0.0_VideoText_previous.gif'

os.makedirs('a_cup_of_coffee_being_poured_w', exist_ok=True)

generate_gif_figure(our_frames_grid, save_name='a_cup_of_coffee_being_poured_w/our_frames_grid.png', n=4, spacing=False)
generate_gif_figure(chatgpt_frames_grid, save_name='a_cup_of_coffee_being_poured_w/chatgpt_frames_grid.png', n=4, spacing=False)
generate_gif_figure(chatgpt_frames_no_grid, save_name='a_cup_of_coffee_being_poured_w/chatgpt_frames_no_grid.png', n=4, spacing=False)
generate_gif_figure(our_frames_no_grid, save_name='a_cup_of_coffee_being_poured_w/our_frames_no_grid.png', n=4, spacing=False)


